In [ ]:
# standard imports
import numpy as np
import math
import scipy
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh

# utility functions
from utilities_modified import *

# Setting Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,[0.5,0])

In [ ]:
# variables
n = mesh.n
N = mesh.N

In [ ]:
# node coordinates
x_node = mesh.points[:,0]
y_node = mesh.points[:,1]

# cell center coordinates
x_cell = np.empty(mesh.N)
y_cell = np.empty(mesh.N)

for cel in mesh.cells:
    i = cel.index
    [x_cell[i],y_cell[i]] = cel.center

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_inc_air/restart/"
data_dict = read_csv_data(data_dir,6000,2,180)

In [ ]:
data_dict.keys()

# POD Decomposition

In [ ]:
q = np.vstack([data_dict["Velocity_x"],data_dict["Velocity_y"],data_dict["Pressure_Coefficient"]])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

In [ ]:
d = int(q.shape[0]/3)
plot_data = q_avg[:d]
plot_cylinder_data(x_node,y_node,plot_data,zoom=True)

In [ ]:
# construction of correlation tensor C
C = np.empty((q.shape[1],q.shape[1]))
for i in tqdm(range(q.shape[1])):
    for j in range(i,q.shape[1],1):
        C[i,j] = 1/(q.shape[1]) * np.inner(q_fluc[:,i],q_fluc[:,j])
        C[j,i] = C[i,j]

In [ ]:
S, V = np.linalg.eigh(C)
S = np.flip(S,0)
V = np.flip(V,1)

In [ ]:
pod_modes = np.zeros((q.shape[0],q.shape[1]))
for i in range(q.shape[1]):
    pod_modes += np.outer(q_fluc[:,i],V[i,:])

In [ ]:
plot_cylinder_data(x_node,y_node,pod_modes[:d,0])

In [ ]:
coeffs = np.matmul(np.linalg.pinv(pod_modes),q_fluc)

In [ ]:
# reconstruction threshold
epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1

num = 5

In [ ]:
velocity_modes = pod_modes[:2*d,:]

## Galerkin System

In [ ]:
def inner_prod(vec1,vec2,weights=None):
    # check for custom weights
    if weights is not None:
        # check for matching dimensions
        if len(vec1)!=len(vec2) or len(vec1)!=len(weights):
            raise ValueError("Dimensions of vectors not matchinh!")
        return np.sum(vec1*vec2*weights) # return weighted sum
    else: 
        return np.sum(vec1*vec2) # return equally weighted sum

In [ ]:
def convection(mesh,q1,q2,d,derivative_method):
    
    # data assessment 
    u1 = q1[:d]
    v1 = q1[d:2*d]
    u2 = q2[:d]
    v2 = q2[:2*d]
    
    # derivative computation
    u2x, u2y = derivative_method(mesh,u2,output=True)
    v2x, v2y = derivative_method(mesh,v2,output=True)

    # construct output 
    if len(q1)/2==d:
        return np.hstack((np.add(u1*u2x,v1*u2y),np.add(u1*v2x,v1*v2y)))
    else:
        return np.hstack((np.add(u1*u2x,v1*u2y),np.add(u1*v2x,v1*v2y),np.zeros(d)))

def diffusion(mesh,q,d,laplacian_method):
    
    # data assessment 
    u = q[:d]
    v = q[d:2*d] 

    # laplacian computation
    _,_,u_lap = laplacian_method(mesh,u,second=True,output=True)
    _,_,v_lap = laplacian_method(mesh,v,second=True,output=True)

    # construct output 
    if len(q)/2==d:
        return np.hstack((u_lap,v_lap))
    else:
        return np.hstack((u_lap,v_lap,np.zeros(d)))

def pressure_grad(mesh,q,d,derivative_method):
    if len(q)==3*d:
        p = q[2*d:]
    else:
        p = q

    # derivative computation
    px,py = derivative_method(p)

    # construct output 
    return hstack((px,py))

In [ ]:
def finite_differences(mesh,data,fd=False,second=False,output=False):
    
    # data vector with length n
    for nod in mesh.nodes:
        
        # stencil indizes
        i = nod.index
        r = nod.get_r()
        l = nod.get_l()
        u = nod.get_u() if nod.get_u() else None  # special boundary case
        b = nod.get_b() if nod.get_b() else None  # special boundary case

        # transform derivatives
        drdx = nod.x/nod.rad
        dthetadx = -nod.y/(nod.rad**2)
        drdy = nod.y/nod.rad
        dthetady = nod.x/(nod.rad**2)     

        # compute cylindrical derivatives
        rad_u = mesh.nodes[u].rad if (u) else None
        rad_b = mesh.nodes[b].rad if (b) else None
        theta_l = mesh.nodes[l].theta if (mesh.nodes[l].theta!=0) else 2*np.pi
        theta_r = mesh.nodes[r].theta if (mesh.nodes[i].theta!=0) else mesh.nodes[r].theta-2*np.pi
        
        if not (u) or not (b):
            dr = 0
            dtheta = 0
        else:
            if fd:
                dr = (data[u]-data[i])/(rad_u-nod.rad)
                dtheta = (data[l]-data[i])/(theta_l-nod.theta)
            else:
                dr = (data[u]-data[b])/(rad_u-rad_b)
                dtheta = (data[l]-data[r])/(theta_l-theta_r)

        # transform to cartesian coordinates
        nod.dx = dtheta*dthetadx + dr*drdx
        nod.dy = dtheta*dthetady + dr*drdy

        if second:
            if not (u) or not (b):
                ddr = 0
                ddtheta = 0
            else:
                ddtheta = (data[l]-2*data[i]+data[r])/((theta_l-nod.theta)*(nod.theta - theta_r)) 
                ddr = (data[u]*(nod.rad-rad_b)+data[b]*(rad_u-nod.rad)-data[i]*(rad_u-rad_b))/((rad_u-nod.rad)*(nod.rad-rad_b)*(rad_u - rad_b)/2)
            
            nod.laplacian = ddr + (1/nod.rad)*dr+(1/(nod.rad**2))*ddtheta

    if output:
        dx = np.empty(mesh.n)
        dy = np.empty(mesh.n)
        if second:
            laplacian = np.empty(mesh.n)
        for nod in mesh.nodes:
            dx[nod.index] = nod.dx
            dy[nod.index] = nod.dy
            if second:
                laplacian[nod.index] = nod.laplacian
        if second:
            return [dx, dy, laplacian]
        else:
            return [dx, dy]

In [ ]:
# computing average pressure gradient
dp = np.zeros_like(q)
for i in tqdm(range(q.shape[1])):
    p_tmp = q[2*d:,i]
    dp[:d,i],dp[d:2*d,i] = finite_differences(mesh,p_tmp,output=True)

dp_avg = np.mean(dp,1)[:2*d]

In [ ]:
U_avg = q_avg[:2*d]

In [ ]:
Re = 100

A = np.empty(num)
B = np.empty((num,num))
C = np.empty((num,num,num))

# additonal terms for pressure method and ergetic conervation
D = np.empty((num,num)) # used if pressure term is considered pressure term 
H = np.zeros(num) # corrector for energetic conservation -> energetic residual 

# initializing temporary operators
conv_avg = convection(mesh,U_avg,U_avg,d,finite_differences)
diff_avg = diffusion(mesh,U_avg,d,finite_differences)
conv_tmp = np.empty((num,2*d))
conv_tmp2 = np.empty((num,2*d))
diff_tmp = np.empty((num,2*d))

# computing NS Operators
for i in range(num):
   conv_tmp[i] = convection(mesh,velocity_modes[:,j],U_avg,d,finite_differences)
   conv_tmp2[i] = convection(mesh,U_avg,velocity_modes[:,j],d,finite_differences)
   diff_tmp[i] = diffusion(mesh,velocity_modes[:,j],d,finite_differences)

## computation of coefficients
for i in tqdm(range(num)):
    A[i] = - inner_prod(velocity_modes[:,i],conv_avg) + 1 / Re * inner_prod(velocity_modes[:,i],diff_avg)
    for j in range(num):
        B[i,j] = - inner_prod(velocity_modes[:,i],conv_tmp[i]) - inner_prod(velocity_modes[:,i],conv_tmp2[i]) + 1 / Re * inner_prod(velocity_modes[:,i],diff_tmp[i])
        for k in range(num):
            C[i,j,k] = - inner_prod(velocity_modes[:,i],convection(mesh,velocity_modes[:,j],velocity_modes[:,k],d,finite_differences))

# Solving ODE system

In [ ]:
# initial conditions for the activations are obtained as activations on the initial timeframe of the data matrix
a0 = coeffs[:num,0] #initial conditions
dt = 1
t0 = 0
tmax = 40000
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))

# definintg galerkin system for ode solver
def galerkin_system(t,a):

    # set global variables references
    global A,B,C
    
    a_dot = np.empty_like(a)
    
    for i in range(a_dot.shape[0]):
        a_dot[i] = A[i] + inner_prod(B[i],a) + np.matmul(np.matmul(np.expand_dims(a,1).T,C[i]),np.expand_dims(a,1))
    return a_dot

In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol

In [ ]:
# Galerkin model based on discrete reconstruction as q = \bar{q}+\sum_i^n a_i(t)*\phi_i(x)
rows = int(num/2)
fig,ax = plt.subplots(rows,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span

# add plots over full time domain
for i in range(num):
    if num <= 2:
        ax[int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))